In [1]:
import math
import random
import numpy as np

In [8]:
def get_obj(sol):
    return round(sum([ x**2 for x in sol]),2)

def get_uniform(search_space):
    sol = []
    for i in range(len(search_space)):
        sol.append(round(random.uniform(search_space[i][0],search_space[i][1]),4))
    return sol

def generate_random_direction(problem_size):
    bounds = [ [-1,1] for i in range(problem_size)]
    return get_uniform(bounds)

In [69]:
def compute_cell_interaction(cell, cells, d, w):
    sum_ = 0 
    for other_cell in cells:
        diff = 0
        for i,v in enumerate(cell['vector']):
            diff += (cell['vector'][i]- other_cell['vector'][i])**2.0
        sum_ += d * math.exp(w * diff)
    return sum_
def attract_repel(cell, cells, d_attr, w_attr, h_rep, w_rep):
    attract = compute_cell_interaction(cell, cells, -d_attr, -w_attr)
    repel = compute_cell_interaction(cell, cells, h_rep, -w_rep)
    return attract + repel

In [70]:
def evaluate(cell, cells, d_attr, w_attr, h_rep, w_rep):
    cell['cost'] = get_obj(cell['vector'])
    cell['inter'] = attract_repel(cell, cells, d_attr, w_attr, h_rep, w_rep)
    cell['fitness'] = cell['cost'] + cell['inter']

In [71]:
def tumble_cell(search_space, cell, step_size):
    step = generate_random_direction(len(search_space))
    vector = []
    for i in range(len(search_space)):
        v = cell['vector'][i] + step_size * step[i]
        if v < search_space[i][0]:
            v = search_space[i][0] 
        if v > search_space[i][1]:
            v = search_space[i][1] 
        vector.append(v)
  
    return {'vector':vector}

In [72]:
def chemotaxis(cells, search_space, chem_steps, swim_length, step_size,
    d_attr, w_attr, h_rep, w_rep):
    best = None
    for j in range(chem_steps):
        moved_cells = []
        for i,cell in enumerate(cells):
            sum_nutrients = 0.0
            evaluate(cell, cells, d_attr, w_attr, h_rep, w_rep)
            if not best or cell['cost'] < best['cost']:
                best = cell
            sum_nutrients += cell['fitness']
            for m in range(swim_length):
                new_cell = tumble_cell(search_space, cell, step_size)
                evaluate(new_cell, cells, d_attr, w_attr, h_rep, w_rep)
                if cell['cost'] < best['cost']:
                    best = cell 
                if new_cell['fitness'] > cell['fitness']:
                    break 
                cell = new_cell
                sum_nutrients += cell['fitness']
            cell['sum_nutrients'] = sum_nutrients
            moved_cells.append(cell)
        cells = moved_cells
    return [best, cells]

In [82]:
def search(search_space, pop_size, elim_disp_steps, repro_steps,
    chem_steps, swim_length, step_size, d_attr, w_attr, h_rep, w_rep,
    p_eliminate):
    # 初始化pop,best
    cells = []
    for i in range(pop_size):
        cells.append({'vector':get_uniform(search_space)})
    best = None
    for l in range(elim_disp_steps):
        for k in range(repro_steps):
            c_best, cells = chemotaxis(cells, search_space, chem_steps,swim_length, step_size, d_attr, w_attr, h_rep, w_rep)
            if not best or c_best['cost'] < best['cost']:
                best = c_best
            cells.sort(key = lambda k:k['sum_nutrients'])
            cells = cells[:int(pop_size/2)] + cells[:int(pop_size/2)]
        for cell in cells:
            if random.random() <= p_eliminate:
                cell['vector'] = get_uniform(search_space)
    return best

In [74]:
# problem configuration
problem_size = 2
search_space = [ [-5,5] for i in range(problem_size)]
# algorithm configuration
pop_size = 50
step_size = 0.1 # Ci
elim_disp_steps = 1 # Ned
repro_steps = 4 # Nre
chem_steps = 70 # Nc
swim_length = 4 # Ns
p_eliminate = 0.25 # Ped
d_attr = 0.1
w_attr = 0.2
h_rep = d_attr
w_rep = 10
# execute the algorithm

In [83]:
search(search_space, pop_size, elim_disp_steps, repro_steps,chem_steps, swim_length, step_size, d_attr, w_attr, h_rep, w_rep,
p_eliminate)

{'vector': [-0.03678000000000001, -0.056829999999999894],
 'cost': 0.0,
 'inter': -1.2572460418738405,
 'fitness': -1.2572460418738405,
 'sum_nutrients': -1.2044260514881577}